<a href="https://colab.research.google.com/github/arizzi/NNTutorial/blob/master/Jet_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
if os.path.isfile('jetImage_Merged.h5') :
    print ("File already downloaded")
else:
    !wget http://cern.ch/arizzi/jetImage_Merged.h5

In [0]:
ls -l


# Dataset Exploration

In this notebook, we explore the input file that we will use across all the exercises.<br>
The file consists of a set of jets, coming from different kinds of particles
- light quarks
- gluons
- W bosons decaying to two light quarks
- Z bosons decaying to two light quarks
- top quarks decaying to three light quarks (via W+b decays)
For each jet, we store the information of the shower it generates, in different formats:
- a list of its constituents (up to 188). Each constituent is represented by an array of 16 quantities (e.g., particle energy and direction, absolute and relative to the jet axis)
- a list of features quantifying the jet kinematic (energy and direction) and the collective shape of its constituents (features quantifying how many prongs the jetc contains)
- the image of the jet, i.e. the temperature map of the energy flow (technically, the transverse momentum) as a function of the angular distance from the jet axis

The notebook is structured as follows:
- We start with basic imports
- We then open the particle dataset, look at it and make some plot
- You are then asked to do the same with the jet dataset
- We then make the plots of the jet images

In [0]:
import h5py
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

In [0]:
%matplotlib inline

---

We now open the input file and look at what's inside

---

In [0]:
f = h5py.File("jetImage_Merged.h5")

In [0]:
print(list(f.keys()))

---

The different datasets correspond to:
- **jetConstituentList**: list of jet consituents' features
- **jetImage**: image of the jet transverse momentum flow
- **jetImage**: image of the jet transverse momentum flow for electrons and photons
- **jetImage**: image of the jet transverse momentum flow for hadrons
- **jets**: list of jet features

---

In [0]:
particleFeatureNames = f.get("particleFeatureNames")
print(particleFeatureNames.shape)
###
jetConstituentList = f.get("jetConstituentList")
print(jetConstituentList.shape)

---

The **particleFeatureNames** and **jetFeatureNames** are arrays of strings, containing (in the right order) the names of the features contained in the **jetConstituentList** and **jets** datasets

---

# Histogram of the tranvserse momentum distribution for the jet constituents

In [0]:
print(particleFeatureNames[:])

---

We select the **transverse momentum**, labelled j1_pt (6th column)<br>
We need to flatten it to a 1D array, in order to make a histogram

---

In [0]:
# all particles pT
particlePt = jetConstituentList[:,:,5]
print(particlePt.shape)

In [0]:
particlePt = particlePt.reshape((particlePt.shape[0]*particlePt.shape[1],))
print(particlePt.shape)

---

Let's make a plot

---

In [0]:
plt.hist(particlePt, bins=50)
plt.gca().set_yscale("log")
yAxis = plt.gca()
plt.xlabel("particle $p_T$ [GeV]")
plt.ylabel("Number of particles")
plt.show()

---

The spike at 0 corresponds to the 0-padding: when a jet has less tha 188 entries (the maximum in the dataset) the list is extended up to 188 by adding zeros. <br>
The rest is the actual distribution, that goes from a few GeV to 1.4 TeV

## **Exercise**
Take the jet dataset and plot the jet pT

---

In [0]:
jetFeatureNames = f.get("jetFeatureNames")
print(jetFeatureNames[:])

In [0]:
jets = f.get("jets")
print(jets.shape)
JetpT = jets[:,1]

In [0]:
plt.hist(JetpT, bins=50)
plt.gca().set_yscale("log")
yAxis = plt.gca()
plt.xlabel("particle $p_T$ [GeV]")
plt.ylabel("Number of particles")
plt.show()

# The true labels

The last 6 features (**'j_g' 'j_q' 'j_w' 'j_z' 'j_t' 'j_undef'**) carry the information of which jet we are looking at. The information is given with a one-hot encoding: all flags are 0 except that the one for the right category, which is set to 1. <br>
For instance, a **W-jet** will have a indices **[0,0,1,0,0,0]**<br>
Notice that the last flag (**undefined**) is used for jets of categories other than the five specified. Since our dataset has only jets of these five kinds, this flag is always set to 0 and can be discarded.

---

## **Exercise**: 
store the 5 interesting flags into a 95500x5 array called y
    
---

In [0]:
y = jets[:,-6:-1]

In [0]:
print(y)

# Average jet images 

In [0]:
jetImage = f.get("jetImage")
jetImageECAL = f.get("jetImageECAL")
jetImageHCAL = f.get("jetImageHCAL")

In [0]:
print(jetImage.shape, jetImageECAL.shape, jetImageHCAL.shape)

---

We now create the average image

---

In [0]:
SUM_jetImageE = np.sum(jetImage, axis=0)
print(SUM_jetImageE.shape)

In [0]:
from matplotlib.colors import LogNorm
plt.imshow(SUM_jetImageE/float(jetImage.shape[0]), origin='lower',norm=LogNorm(vmin=0.01))
plt.colorbar()
plt.show()

---

## **Exercise**: 
Do the same for ECAL and HCAL images
    
---

In [0]:
SUM_jetImageE = np.sum(jetImageECAL, axis=0)
plt.imshow(SUM_jetImageE/float(jetImage.shape[0]), origin='lower',norm=LogNorm(vmin=0.01))
plt.colorbar()
plt.show()

In [0]:
SUM_jetImageE = np.sum(jetImageHCAL, axis=0)
plt.imshow(SUM_jetImageE/float(jetImage.shape[0]), origin='lower',norm=LogNorm(vmin=0.01))
plt.colorbar()
plt.show()

---

**NOTICE THAT** the jet momentum is mainly carried by hadrons (as it should be.
The electrons and photons have very small energy, but they might carry some information on the jets. So, it might be useful to use the two image "layers" separetly. As a default, we will not explore this option and just take a simple 1-layer image with the total energy

---